In [1]:
import pandas as pd
import osmnx as ox
import networkx as nx
import random
import math
import folium
import os
import time
import numpy as np

In [2]:
# --- KONFIGURASI ---
filename = '../Dataset/jne_solo.csv'    # Pastikan file ini ada
nama_folder_output = "../Result"        # Folder output
KAPASITAS_TRUK = 200                    # Kapasitas Truk Canter (Paket)
TARGET_TOTAL_PAKET = 400                # Total paket dummy

# KOORDINAT HARDCODE JNE WAREHOUSE ADI SUMARMO (COLOMADU)
# (Sekitar Jl. Adi Sumarmo dekat Exit Tol/Bandara)
FIXED_WAREHOUSE_LAT = -7.532519372469423 
FIXED_WAREHOUSE_LON = 110.76423789494254 

print("✅ Cell 1: Konfigurasi Loaded (Warehouse: JNE Adi Sumarmo).")

✅ Cell 1: Konfigurasi Loaded (Warehouse: JNE Adi Sumarmo).


In [3]:
# ==========================================
# LOAD DATA & GENERATE DUMMY PAKET
# ==========================================
# 1. LOAD DATA AGEN
print("\n[Loading Data Agen...]")
try:
    df_jne = pd.read_csv(filename, sep=';')
    df_jne[['lat', 'lon']] = df_jne['Koordinat'].str.split(',', expand=True).astype(float)
    
    # Override lokasi Warehouse dengan input manual (Adi Sumarmo)
    warehouse_lat = FIXED_WAREHOUSE_LAT
    warehouse_lon = FIXED_WAREHOUSE_LON
    
    print(f"✅ {len(df_jne)} Agen Loaded.")
    print(f"📍 Warehouse FIXED: JNE Adi Sumarmo ({warehouse_lat}, {warehouse_lon})")

except Exception as e:
    print(f"❌ Error: {e}")

# 2. GENERATE DUMMY PAKET
print("\n[Generating Dummy Paket...]")
all_paket_data = []
paket_per_agen = TARGET_TOTAL_PAKET // len(df_jne) 
sisa_paket = TARGET_TOTAL_PAKET % len(df_jne)
counter_id = 1

def get_random_point_around(lat, lon, radius_km):
    r = radius_km / 111.0
    u = random.random()
    v = random.random()
    w = r * math.sqrt(u)
    t = 2 * math.pi * v
    x = w * math.cos(t)
    y_offset = w * math.sin(t)
    return lat + y_offset, lon + (x / math.cos(math.radians(lat)))

for idx, row in df_jne.iterrows():
    jatah = paket_per_agen + (1 if idx < sisa_paket else 0)
    for _ in range(jatah):
        new_lat, new_lon = get_random_point_around(row['lat'], row['lon'], 2.5)
        all_paket_data.append({
            'ID Paket': f'PKT-{counter_id:04d}',
            'lat': new_lat,
            'lon': new_lon,
            'Agen Induk': row['Nama JNE'],
            'Agen Lat': row['lat'],
            'Agen Lon': row['lon']
        })
        counter_id += 1

df_paket = pd.DataFrame(all_paket_data)
print(f"✅ Cell 2: {len(df_paket)} Paket Generated.")


[Loading Data Agen...]
✅ 48 Agen Loaded.
📍 Warehouse FIXED: JNE Adi Sumarmo (-7.532519372469423, 110.76423789494254)

[Generating Dummy Paket...]
✅ Cell 2: 400 Paket Generated.


In [4]:
# ==========================================
# 3. DOWNLOAD PETA (FULL KARESIDENAN SURAKARTA)
# ==========================================
print("\n[Downloading Map: Karesidenan Surakarta...]")
wilayah_solo_raya = [
    "Surakarta, Central Java, Indonesia",
    "Sukoharjo, Central Java, Indonesia",
    "Karanganyar, Central Java, Indonesia", 
    "Boyolali, Central Java, Indonesia",
    "Klaten, Central Java, Indonesia",
    "Sragen, Central Java, Indonesia",
    "Wonogiri, Central Java, Indonesia"
]

try:
    # Download graph gabungan
    G = ox.graph_from_place(wilayah_solo_raya, network_type='drive')
    print(f"✅ Cell 3: Peta Berhasil Didownload! ({len(G.nodes)} Simpul)")
except Exception as e:
    print(f"❌ Gagal download peta: {e}")


[Downloading Map: Karesidenan Surakarta...]
✅ Cell 3: Peta Berhasil Didownload! (322827 Simpul)


In [5]:
# ==========================================
# 4. HITUNG RUTE (REAL ROAD DISTANCE)
# ==========================================
print("\n[Cell 4: Menghitung Rute 2 Truk (Parallel)...]")

# --- A. RUTE TRUK (CVRP: Multi-Vehicle) ---
print("👉 Menghitung Rute Distribusi 2 Truk...")

# Kita siapkan struktur data untuk 2 Truk
# Format: [ {'path': [nodes], 'agents': [info_agen]}, ... ]
data_armada_truk = [] 

node_warehouse = ox.nearest_nodes(G, FIXED_WAREHOUSE_LON, FIXED_WAREHOUSE_LAT)
demand_per_agen = df_paket['Agen Induk'].value_counts().to_dict()

# List semua agen yang perlu dikunjungi
agen_to_visit = []
for idx, row in df_jne.iterrows():
    node = ox.nearest_nodes(G, row['lon'], row['lat'])
    demand = demand_per_agen.get(row['Nama JNE'], 0)
    if demand > 0:
        agen_to_visit.append({
            'Nama': row['Nama JNE'], 
            'Node': node, 
            'Demand': demand, 
            'lat': row['lat'], 
            'lon': row['lon']
        })

# ALGORITMA ALOKASI TRUK (Sequential Greedy)
# Kita jalankan loop untuk mengisi Truk 1 dulu sampai penuh/selesai, 
# sisanya otomatis jadi jatah Truk 2.
unvisited_agen = agen_to_visit.copy()
jumlah_truk_aktif = 2 # Kita set 2 Truk

for i in range(jumlah_truk_aktif):
    nama_truk = f"Truk {i+1}"
    print(f"\n   🚛 Memulai Rute {nama_truk}...")
    
    current_capacity = KAPASITAS_TRUK
    # Agar aman, kita tambah sedikit kapasitas toleransi (buffer) 
    # supaya semua paket terangkut cuma oleh 2 truk
    current_capacity += 20 
    
    current_node = node_warehouse
    path_nodes = [node_warehouse]
    agents_visited_this_truck = []
    visit_order = 1
    
    # Loop pengisian satu truk
    while unvisited_agen:
        target_agen = None
        idx_remove = -1
        is_at_warehouse = (current_node == node_warehouse)
        
        # Logika: Truk 1 ambil yang PALING JAUH (Outer Ring)
        # Truk 2 ambil sisanya (Inner Ring/Area lain)
        best_metric = -1.0 if is_at_warehouse else float('inf')

        for idx, agen in enumerate(unvisited_agen):
            if agen['Demand'] <= current_capacity:
                try:
                    dist = nx.shortest_path_length(G, current_node, agen['Node'], weight='length')
                    if is_at_warehouse:
                        # Cari start point terjauh
                        if dist > best_metric:
                            best_metric = dist; target_agen = agen; idx_remove = idx
                    else:
                        # Cari tetangga terdekat
                        if dist < best_metric:
                            best_metric = dist; target_agen = agen; idx_remove = idx
                except: continue
        
        if target_agen:
            path_nodes.append(target_agen['Node'])
            current_node = target_agen['Node']
            current_capacity -= target_agen['Demand']
            unvisited_agen.pop(idx_remove)
            
            agents_visited_this_truck.append({
                'name': target_agen['Nama'],
                'lat': target_agen['lat'],
                'lon': target_agen['lon'],
                'order': visit_order,
                'demand': target_agen['Demand']
            })
            visit_order += 1
            print(f"      -> {nama_truk} angkut {target_agen['Demand']} paket ke {target_agen['Nama']}")
        else:
            # Tidak ada agen yang cocok/muat lagi untuk truk ini
            break
            
    # Tutup rute truk ini (Balik Gudang)
    if path_nodes[-1] != node_warehouse:
        path_nodes.append(node_warehouse)
        
    # Generate Geometri Jalan (Garis)
    segments = []
    for k in range(len(path_nodes) - 1):
        try:
            seg = nx.shortest_path(G, path_nodes[k], path_nodes[k+1], weight='length')
            if k > 0 and len(segments) > 0: seg = seg[1:]
            segments.extend(seg)
        except: pass
        
    # Simpan Data Truk Ini
    data_armada_truk.append({
        'nama': nama_truk,
        'path_segments': segments,
        'visited_agents': agents_visited_this_truck
    })

# --- B. RUTE KURIR (Tidak Berubah) ---
print("\n👉 Menghitung Rute Kurir (TSP)...")
rute_kurir_final = {}
paket_orders = {}

grouped = df_paket.groupby('Agen Induk')
for i, (nama, group) in enumerate(grouped):
    node_agen = ox.nearest_nodes(G, group.iloc[0]['Agen Lon'], group.iloc[0]['Agen Lat'])
    unvisited = [{'node': ox.nearest_nodes(G, r['lon'], r['lat']), 'id': r['ID Paket'], 'lat': r['lat'], 'lon': r['lon']} for _, r in group.iterrows()]

    curr = node_agen; path_seq = [node_agen]
    agent_pkt_orders = []; order = 1
    
    while unvisited:
        nearest = None; min_d = float('inf'); idx_rem = -1
        for idx, pkt in enumerate(unvisited):
            try:
                d = nx.shortest_path_length(G, curr, pkt['node'], weight='length')
                if d < min_d: min_d = d; nearest = pkt; idx_rem = idx
            except: continue
        if nearest:
            path_seq.append(nearest['node']); curr = nearest['node']
            unvisited.pop(idx_rem)
            agent_pkt_orders.append({'id': nearest['id'], 'order': order, 'lat': nearest['lat'], 'lon': nearest['lon']})
            order += 1
        else: break
    path_seq.append(node_agen)
    paket_orders[nama] = agent_pkt_orders
    
    full_path = []
    for k in range(len(path_seq)-1):
        try:
            seg = nx.shortest_path(G, path_seq[k], path_seq[k+1], weight='length')
            if k > 0: seg = seg[1:]
            full_path.extend(seg)
        except: pass
    rute_kurir_final[nama] = full_path

print("✅ Cell 4: Selesai (2 Rute Truk Terpisah).")


[Cell 4: Menghitung Rute 2 Truk (Parallel)...]
👉 Menghitung Rute Distribusi 2 Truk...

   🚛 Memulai Rute Truk 1...
      -> Truk 1 angkut 8 paket ke JNE Agen Bersaudara
      -> Truk 1 angkut 8 paket ke Agen JNE Kumbolo
      -> Truk 1 angkut 8 paket ke JNE 048 Papahan
      -> Truk 1 angkut 8 paket ke Cabang JNE Karanganyar
      -> Truk 1 angkut 8 paket ke JNE Mayor Ahmadi
      -> Truk 1 angkut 8 paket ke JNE Agen 075
      -> Truk 1 angkut 9 paket ke JNE Mojolaban
      -> Truk 1 angkut 8 paket ke JNE Agen 23 Pasar Kliwon
      -> Truk 1 angkut 9 paket ke JNE Pasar Klewer
      -> Truk 1 angkut 9 paket ke JNE Agen Gading
      -> Truk 1 angkut 8 paket ke JNE Agen 064
      -> Truk 1 angkut 9 paket ke JNE Citra Damai
      -> Truk 1 angkut 9 paket ke JNE Gubug Mas 016
      -> Truk 1 angkut 9 paket ke JNE Express Slamet Riyadi
      -> Truk 1 angkut 9 paket ke JNE Express 031
      -> Truk 1 angkut 9 paket ke JNE Cantika
      -> Truk 1 angkut 9 paket ke JNE Gloria
      -> Truk 1 

In [6]:
# ==========================================
# 5. VISUALISASI FOLIUM
# ==========================================
from folium.features import DivIcon

print("\n[Cell 5: Visualisasi Multi-Layer Truk...]")
m = folium.Map(location=[FIXED_WAREHOUSE_LAT, FIXED_WAREHOUSE_LON], zoom_start=11, tiles='OpenStreetMap')

# Marker Warehouse (Permanen, tidak masuk layer toggle biar selalu muncul)
folium.Marker(
    [FIXED_WAREHOUSE_LAT, FIXED_WAREHOUSE_LON],
    popup="<b>WAREHOUSE PUSAT</b>",
    icon=folium.Icon(color='black', icon='plane', prefix='fa')
).add_to(m)

# KONFIGURASI WARNA TRUK
warna_truk = {
    'Truk 1': 'red',   # Truk 1 Merah
    'Truk 2': 'blue'   # Truk 2 Biru
}

# --- LOOP MEMBUAT LAYER UNTUK SETIAP TRUK ---
for data_truk in data_armada_truk:
    nama_truk = data_truk['nama']
    warna = warna_truk.get(nama_truk, 'purple')
    
    # Bikin Layer Group per Truk (Default: Show=True)
    layer_truk = folium.FeatureGroup(name=f"🚛 Jalur {nama_truk}", show=True)
    
    # 1. Gambar Garis Rute
    if data_truk['path_segments']:
        coords = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in data_truk['path_segments']]
        folium.PolyLine(
            coords, 
            color=warna, 
            weight=5, 
            opacity=0.8, 
            tooltip=f"Rute {nama_truk}"
        ).add_to(layer_truk)
        
    # 2. Gambar Marker Agen (Angka Urut)
    for agent in data_truk['visited_agents']:
        # CSS untuk lingkaran angka
        icon_html = f"""
            <div style="
                font-size: 12pt; color: white; background-color: {warna};
                border: 2px solid white; border-radius: 50%;
                text-align: center; line-height: 30px; 
                width: 30px; height: 30px;
                box-shadow: 2px 2px 5px rgba(0,0,0,0.5);
                font-weight: bold;">
                {agent['order']}
            </div>
        """
        
        folium.Marker(
            location=[agent['lat'], agent['lon']],
            popup=f"<b>{agent['name']}</b><br>Diantar oleh: {nama_truk}<br>Urutan: {agent['order']}",
            tooltip=f"{nama_truk} - No. {agent['order']}",
            icon=DivIcon(icon_size=(30,30), icon_anchor=(15,15), html=icon_html)
        ).add_to(layer_truk)
        
    # Masukkan layer truk ke peta
    layer_truk.add_to(m)


# --- LAYER KURIR (Tetap sama) ---
for nama, nodes in rute_kurir_final.items():
    layer_kurir = folium.FeatureGroup(name=f"🛵 Kurir: {nama}", show=False)
    if nodes:
        coords = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in nodes]
        folium.PolyLine(coords, color='#32CD32', weight=3, opacity=0.8).add_to(layer_kurir)
        start = nodes[0]
        folium.Marker([G.nodes[start]['y'], G.nodes[start]['x']], popup=f"Base: {nama}", icon=folium.Icon(color='green', icon='motorcycle', prefix='fa')).add_to(layer_kurir)

    if nama in paket_orders:
        for pkg in paket_orders[nama]:
            html_pkg = f'<div style="font-size: 8pt; color: white; background-color: #32CD32; border: 1px solid white; border-radius: 50%; text-align: center; line-height: 18px; width: 18px; height: 18px;">{pkg["order"]}</div>'
            folium.Marker([pkg['lat'], pkg['lon']], popup=pkg['id'], icon=DivIcon(icon_size=(18,18), html=html_pkg)).add_to(layer_kurir)
            
    layer_kurir.add_to(m)

folium.LayerControl(collapsed=False).add_to(m)
print("✅ Cell 5: Peta Siap! (Cek Sidebar: Ada Opsi Truk 1 & Truk 2)")


[Cell 5: Visualisasi Multi-Layer Truk...]
✅ Cell 5: Peta Siap! (Cek Sidebar: Ada Opsi Truk 1 & Truk 2)


In [7]:
# ==========================================
# 6. SIMPAN OUTPUT
# ==========================================
# Setup Folder
base_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()
output_dir = os.path.join(base_dir, nama_folder_output)
if not os.path.exists(output_dir): os.makedirs(output_dir)

# Save HTML
html_path = os.path.join(output_dir, "peta_distribusi_CVRP_Real.html")
m.save(html_path)

print(f"✅ Cell 6: File saved at: {html_path}")

✅ Cell 6: File saved at: d:\Kuliah\Semester 3\Mat Diskrit\Project\Pembagian-Area-Pengiriman-JNE\Src\../Result\peta_distribusi_CVRP_Real.html
